<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/seed1234_neurons_batch1_64_to_384.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
mnist_trainset = torchvision.datasets.MNIST(root='./', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = torchvision.datasets.MNIST(root='./', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=1, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

In [16]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self, neuron_size):
        super(Model, self).__init__()
        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, neuron_size)
        self.linear_2 = torch.nn.Linear(neuron_size, 10)
        self.sigmoid12  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        pred = self.linear_2(x)
        return pred
# ************* modify this section for later use *************

In [17]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [18]:
def sparsity_calculator(final_spareness, neuron_size):
    sparseness_list_avg = list()
    sparseness_list_std = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, neuron_size))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)

        mean_sparseness_per_epoch = torch.mean(population_sparseness)
        std_sparseness_per_epoch  = torch.std(population_sparseness)

        sparseness_list_avg.append(mean_sparseness_per_epoch)

    return sparseness_list_avg

In [19]:
def selectivity(hidden_layer_each_neuron, neuron_size):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(neuron_size)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 'neuron_size' lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(neuron_size)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [20]:
# ************* modify this section for later use *************
def avg_std_calculator(_hidden_layer_each_neuron_12, neuron_size):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12, neuron_size)

    final_selectivity_avg = (avg_selectivity12) / 1
    final_selecvitity_std = (std_selectivity12) / 1

    return final_selectivity_avg, final_selecvitity_std
# ************* modify this section for later use *************

In [21]:
def model_factory(optimizer_name, seed_num, neuron_size):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model(neuron_size)
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparsity_avg, selectivity_list_avg, selectivity_list_std = sparsity_selectivity_trainer(optimizer=my_optimizer, model=my_model, neuron_size=neuron_size)

    # ************* modify this section for later use *************
    file_saver = open(f"seed{seed_num}_batch1_neuronsize{neuron_size}_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparsity_avg)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()

    if seed_num == 1:
        if neuron_size == 64:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize64_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize64_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize64_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize64_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 128:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize128_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize128_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize128_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize128_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 256:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize256_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize256_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize256_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize256_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 384:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize384_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize384_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize384_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize384_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 512:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize512_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize512_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize512_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize512_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 640:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize640_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize640_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize640_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize640_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 768:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize768_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize768_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize768_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize768_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

    elif seed_num == 100:
        if neuron_size == 64:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize64_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize64_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize64_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize64_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 128:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize128_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize128_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize128_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize128_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 256:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize256_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize256_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize256_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize256_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 384:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize384_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize384_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize384_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize384_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 512:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize512_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize512_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize512_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize512_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 640:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize640_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize640_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize640_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize640_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 768:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize768_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize768_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize768_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize768_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

    # seed 1234
    elif seed_num == 1234:
        if neuron_size == 64:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize64_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize64_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize64_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize64_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 128:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize128_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize128_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize128_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize128_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 256:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize256_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize256_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize256_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize256_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 384:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize384_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize384_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize384_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize384_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 512:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize512_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize512_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize512_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize512_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 640:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize640_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize640_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize640_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize640_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 768:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize768_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize768_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize768_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize768_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

In [22]:
no_epochs = 30
def sparsity_selectivity_trainer(optimizer, model, neuron_size):

    criterion = torch.nn.CrossEntropyLoss()
    test_acc   = list()

    best_test_loss = 1

    selectivity_avg_list = list()
    selectivity_std_list = list()

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    # ************* modify this section for later use *************

    for epoch in range(no_epochs):

        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(neuron_size)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************

        total_train_loss = 0
        total_test_loss = 0

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        # ************* modify this section for later use *************

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])

            # ************* modify this section for later use *************
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(neuron_size):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])
    
        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12, neuron_size)
        # ************* modify this section for later use *************
        
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)

        # this conains activations for all epochs 
        final_spareness_12.append(hidden_layer_activation_list_12)
        # ***************** sparsity calculation ***************** #

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

    # ***************** sparsity calculation ***************** #
    sparsity_list12_avg = sparsity_calculator(final_spareness_12, neuron_size)

    average_sparsity = list()

    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list12_avg[i].item()) / 1 )
    # ***************** sparsity calculation ***************** #

    return test_acc, average_sparsity, selectivity_avg_list, selectivity_std_list

In [23]:
# Seed 1234 

torch.manual_seed(1234)
np.random.seed(1234)

In [11]:


# 64 neurons 
model_factory('Adadelta', 1234, 64)
model_factory('Adagrad', 1234, 64)
model_factory('SGD', 1234, 64)
model_factory('Adam', 1234, 64)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 19509.33179794, Test Loss: 46.73631245, Test Accuracy: 0.94070000

Epoch: 2/30, Train Loss: 13253.01882433, Test Loss: 38.45893130, Test Accuracy: 0.95030000

Epoch: 3/30, Train Loss: 10946.36862841, Test Loss: 33.72769400, Test Accuracy: 0.95840000

Epoch: 4/30, Train Loss: 9660.02711802, Test Loss: 31.75145441, Test Accuracy: 0.95940000

Epoch: 5/30, Train Loss: 8882.56415675, Test Loss: 29.74532770, Test Accuracy: 0.96430000

Epoch: 6/30, Train Loss: 8274.65386416, Test Loss: 30.00654842, Test Accuracy: 0.96260000

Epoch: 7/30, Train Loss: 7782.90821147, Test Loss: 30.88553090, Test Accuracy: 0.96440000

Epoch: 8/30, Train Loss: 7509.84837562, Test Loss: 29.33320836, Test Accuracy: 

In [12]:
# 128 neurons 
model_factory('Adadelta', 1234, 128)
model_factory('Adagrad', 1234, 128)
model_factory('SGD', 1234, 128)
model_factory('Adam', 1234, 128)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=128, bias=True)
  (linear_2): Linear(in_features=128, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 18444.42383475, Test Loss: 41.77293881, Test Accuracy: 0.94690000

Epoch: 2/30, Train Loss: 11527.90064449, Test Loss: 31.18162055, Test Accuracy: 0.95930000

Epoch: 3/30, Train Loss: 9385.49577610, Test Loss: 28.46026822, Test Accuracy: 0.96500000

Epoch: 4/30, Train Loss: 8025.72175532, Test Loss: 25.62459275, Test Accuracy: 0.96690000

Epoch: 5/30, Train Loss: 7110.99498187, Test Loss: 27.11246686, Test Accuracy: 0.96720000

Epoch: 6/30, Train Loss: 6587.54901018, Test Loss: 24.32751544, Test Accuracy: 0.97120000

Epoch: 7/30, Train Loss: 5977.88481313, Test Loss: 24.99082384, Test Accuracy: 0.97050000

Epoch: 8/30, Train Loss: 5568.07979717, Test Loss: 22.36310472, Test Accuracy:

In [13]:
# 256 neurons 
model_factory('Adadelta', 1234, 256)
model_factory('Adagrad', 1234, 256)
model_factory('SGD', 1234, 256)
model_factory('Adam', 1234, 256)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 18841.70052454, Test Loss: 40.31632494, Test Accuracy: 0.94740000

Epoch: 2/30, Train Loss: 11026.84288938, Test Loss: 30.45795690, Test Accuracy: 0.96080000

Epoch: 3/30, Train Loss: 8372.96893607, Test Loss: 28.84013438, Test Accuracy: 0.96620000

Epoch: 4/30, Train Loss: 6888.41192414, Test Loss: 25.89124378, Test Accuracy: 0.96820000

Epoch: 5/30, Train Loss: 6034.47159533, Test Loss: 20.37249598, Test Accuracy: 0.97420000

Epoch: 6/30, Train Loss: 5262.56894235, Test Loss: 20.02413215, Test Accuracy: 0.97310000

Epoch: 7/30, Train Loss: 4603.04771062, Test Loss: 20.19460536, Test Accuracy: 0.97400000

Epoch: 8/30, Train Loss: 4312.22388952, Test Loss: 19.15625626, Test Accuracy:

In [24]:
# 384 neurons 
model_factory('Adadelta', 1234, 384)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=384, bias=True)
  (linear_2): Linear(in_features=384, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 19563.48843208, Test Loss: 40.67091092, Test Accuracy: 0.95190000

Epoch: 2/30, Train Loss: 10731.72155710, Test Loss: 27.22717592, Test Accuracy: 0.96440000

Epoch: 3/30, Train Loss: 8000.16694686, Test Loss: 29.50317174, Test Accuracy: 0.96340000

Epoch: 4/30, Train Loss: 6661.12701197, Test Loss: 24.32296038, Test Accuracy: 0.96950000

Epoch: 5/30, Train Loss: 5622.16367523, Test Loss: 20.70760627, Test Accuracy: 0.97660000

Epoch: 6/30, Train Loss: 4938.79764466, Test Loss: 20.93197935, Test Accuracy: 0.97620000

Epoch: 7/30, Train Loss: 4377.69068638, Test Loss: 19.86728702, Test Accuracy: 0.97590000

Epoch: 8/30, Train Loss: 3933.82595406, Test Loss: 20.29470902, Test Accuracy:

In [25]:
model_factory('Adagrad', 1234, 384)


my_model: Model(
  (linear_1): Linear(in_features=784, out_features=384, bias=True)
  (linear_2): Linear(in_features=384, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 0.1
    lr_decay: 0
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 11358.17686582, Test Loss: 22.12879748, Test Accuracy: 0.96730000

Epoch: 2/30, Train Loss: 4885.44120730, Test Loss: 18.26265254, Test Accuracy: 0.97190000

Epoch: 3/30, Train Loss: 3264.47297707, Test Loss: 15.91543117, Test Accuracy: 0.97540000

Epoch: 4/30, Train Loss: 2325.83017820, Test Loss: 14.65049615, Test Accuracy: 0.97780000

Epoch: 5/30, Train Loss: 1680.08979093, Test Loss: 14.20706485, Test Accuracy: 0.97790000

Epoch: 6/30, Train Loss: 1270.55998929, Test Loss: 13.58271275, Test Accuracy: 0.97880000

Epoch: 7/30, Train Loss: 969.78730022, Test Loss: 13.44216104, Test Accuracy: 0.97960000

Epoch: 8/30, Train Loss: 757.62102183, Test L

In [26]:
model_factory('SGD', 1234, 384)


my_model: Model(
  (linear_1): Linear(in_features=784, out_features=384, bias=True)
  (linear_2): Linear(in_features=384, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 14204.40901976, Test Loss: 26.43992034, Test Accuracy: 0.95910000

Epoch: 2/30, Train Loss: 6141.79409067, Test Loss: 20.95471618, Test Accuracy: 0.96820000

Epoch: 3/30, Train Loss: 4159.93660309, Test Loss: 16.39044069, Test Accuracy: 0.97500000

Epoch: 4/30, Train Loss: 3251.51370835, Test Loss: 15.57894610, Test Accuracy: 0.97830000

Epoch: 5/30, Train Loss: 2520.75266061, Test Loss: 17.18050077, Test Accuracy: 0.97480000

Epoch: 6/30, Train Loss: 1954.75323372, Test Loss: 17.40113000, Test Accuracy: 0.97450000

Epoch: 7/30, Train Loss: 1393.36917434, Test Loss: 16.17355087, Test Accuracy: 0.97600000

Epoch: 8/30, Train Loss: 994.73826959, Test Loss: 14.622070

In [27]:
model_factory('Adam', 1234, 384)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=384, bias=True)
  (linear_2): Linear(in_features=384, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 12010.91230602, Test Loss: 22.63865483, Test Accuracy: 0.96610000

Epoch: 2/30, Train Loss: 5124.75464130, Test Loss: 17.09695307, Test Accuracy: 0.97520000

Epoch: 3/30, Train Loss: 3678.85543276, Test Loss: 15.29960955, Test Accuracy: 0.97810000

Epoch: 4/30, Train Loss: 2674.09666812, Test Loss: 20.37837027, Test Accuracy: 0.97530000

Epoch: 5/30, Train Loss: 2042.05032691, Test Loss: 16.69328676, Test Accuracy: 0.97980000

Epoch: 6/30, Train Loss: 1646.98364376, Test Loss: 18.93368876, Test Accuracy: 0.97950000

Epoch: 7/30, Train Loss: 1419.22543489, Test Loss: 19.35989850, Test Accuracy: 0.97890000

Epoch: 8/30, Train Loss: 1092.59300848, Test Loss: 